050624 Given that we have the best 2 decoded spots per cell per comb region, now attempt to find the seqential spots. 

Test on `fov = 05`

Idea:
- For each seq round (total 60):
    - for each decoded comb index (total 595):
        - for each cell (total 223):
            - get all fitted spots in the seq round (len M)
            - compute score based on 1) distance to decoded spots 2) brightness similar to ioMicro::get_score_per_colorV2()
            - find nbest spots similar to ioMicroPostfits::keep_best_per_cell_fast()
            - store it?
            

In [1]:
analysis_folder = '/mnt/tscc/wed009/WholeGenome/NMERFISH_Jenny/'
import sys
sys.path.append(analysis_folder)
from ioMicro import *

In [2]:
#### mediator version ###
save_folder =r'/projects/ps-renlab2/wed009/analysis_031524_v1_acry_37C_pilot'
fov='Conv_zscan__05'
set_=''

# load seq round dic
meta_folder = '/mnt/tscc/wed009/WholeGenome/data/FromBlair/V1_pilot_seq'
dic_seq_round = pickle.load(open(os.path.join(meta_folder, 'dic_seq_round.pkl'), 'rb'))

# load chrom abb dic
chrom_fl = os.path.join(analysis_folder, 'dic_chromatic_abberation_Jenny_Scope4_4_17_2024.pkl')
chrom_fl

# begin code insert here
functions pasted from ioMicroPostfits.py

In [79]:
def load_segmentation(fl_ref, save_folder = save_folder):
    H = os.path.basename(os.path.dirname(fl_ref))
    seg_fl = os.path.join(save_folder, 'Segmentation', fov + '--' + H + '--dapi_segm.npz')
    seg = np.load(seg_fl)
    segm = seg['segm']
    shape = seg['shape']
    resc = shape/segm.shape
    return segm, resc

In [84]:
# load the control or seqential fits
# seq R goes from 0-59
def load_seq_fits(seq_bit, 
                  set_ = set_,
                  save_folder = save_folder,
                  fov = fov,
                chrom_fl = r'C:\Scripts\NMERFISH_Jenny\dic_chromatic_abberation_Jenny_Scope4_4_17_2024.pkl'):
#     set_ = self.set_
#     fov = self.fov
#     save_folder = self.save_folder
    drift_fl = save_folder+os.sep+'driftNew_'+fov.split('.')[0]+'--'+set_+'.pkl'
    drifts,all_flds,fov,fl_ref = pickle.load(open(drift_fl,'rb'))
    #self.drifts,self.all_flds,self.fov,self.fl_ref = drifts,all_flds,fov,fl_ref

    XH = []
    # reconstruct tag
    icol = (int(seq_bit[4:])-1) % 3
    irepeat = np.ceil(int(seq_bit[4:]) / 3)
    plate_well_1 = seq_bit[:4] #P2_F
    plate_well_2 = '_'.join([str(int(irepeat*3-i)) for i in range(2,-1, -1)])#7_8_9
    plate_well = plate_well_1  + plate_well_2

    # get H
    iH = [all_flds.index(fld) for fld in all_flds if plate_well in fld][0] + 1 # take only first one; H is 1-indexed
    tag = 'H' + str(iH) + '_' + plate_well
    print(f'Loading tag :{tag}')

    save_fl = save_folder+os.sep+fov.split('.')[0]+'--'+tag+'--col'+str(icol)+'__Xhfits.npz'

    # load fits
    Xh = np.load(save_fl,allow_pickle=True)['Xh']### load the fits
    x3d = Xh[:,:3]
    # chromatic abb
    if chrom_fl is not None:
        dic_chrom = pickle.load(open(chrom_fl,'rb'))
        m = dic_chrom.get('m'+str(icol),None)
        hfactor = dic_chrom.get('hfactor'+str(icol),1)
        x3dT = apply_colorcor(x3d,m=m)
        Xh[:,:3] = x3dT
        Xh[:,-1] = Xh[:,-1]*hfactor
    print(Xh.shape)
    if len(Xh.shape):
        #Xh = Xh[Xh[:,-1]>th_h]
        if len(Xh):
            tzxy = drifts[iH-1][0] # fetching the correct drift files
            Xh[:,:3]+=tzxy# drift correction
            
    # load segmentation
    segm, resc = load_segmentation(fl_ref)
    return Xh, (segm, resc)   

In [118]:
def get_seq_Xhfc(seq_bit, gene_names = gene_names, 
                 h_th = 2000, dic_seq_round = dic_seq_round):
    Xh_seq, (segm,resc) = load_seq_fits(seq_bit, chrom_fl = chrom_fl)
    # filter based on brightness
    Xh_seq = Xh_seq[Xh_seq[:,-3]>h_th]
    # get comb region index
    R = dic_seq_round[seq_bit]
    print(f'seq R: {R}')
    # add two dummy columns
    # in the decoded version, this corresponds to icol(color) and R (bit number)
    Xh_seq = np.concatenate((Xh_seq, np.full((Xh_seq.shape[0], 1), -1)), axis=1) # add icol
    Xh_seq = np.concatenate((Xh_seq, np.full((Xh_seq.shape[0], 1), R)), axis=1) # add seq gene name: round 0-59
    
    # assign to cell
    # should be in the same cell space as the comb
    Xh_seq_red = np.round(Xh_seq[:,:3]/resc).astype(int) # resacle to seg space
    in_image = np.all((Xh_seq_red>0)&(Xh_seq_red<segm.shape),axis=-1)
    cell_ids = np.zeros(len(in_image))
    cell_ids[in_image]=segm[tuple(Xh_seq_red[in_image].T)]
    XHfc_seq = np.concatenate([Xh_seq,cell_ids[:,np.newaxis]],axis=-1)
    # remove any spots in background
    XHfc_seq = XHfc_seq[XHfc_seq[:, -1] > 0]
    
    return XHfc_seq, R

# load filter decoded

In [3]:
filter_decoded_fl = os.path.join(save_folder, f'FilterDecoded_{fov}.npz')
filter_decoded_fl

'/projects/ps-renlab2/wed009/analysis_031524_v1_acry_37C_pilot/FilterDecoded_Conv_zscan__05.npz'

In [38]:
fd = np.load(filter_decoded_fl)
XHfc = fd['XHfc']
XHfc_best = fd['XHfc_best']
gene_names = fd['gene_names']
gene_indices = fd['gene_indices']

In [12]:
XHfc_best.shape

(327798, 11)

In [16]:
# metadata
XHfc_best[:, -1] # -1: cell id
XHfc_best[:, -2] # -2: gene indices (need to exclude non blank genes) in gene_names
XHfc_best[:, -3] # -3: score_A
XHfc_best[:, -4] # -4: normalized intensity

array([4314.33056641, 4782.37272135, 5288.46940104, ..., 6973.93066406,
       4957.27742513, 6648.24869792])

In [46]:
XHfc_best[:, -2]

array([ 127.,  128.,  129., ..., 1188., 1188., 1189.])

In [49]:
# filter only genes non-blank
is_blank = np.array(['blank' in e for e in gene_names[XHfc_best[:, -2].astype(int)]])
XHfc_best_valid = XHfc_best[~is_blank]
XHfc_best_valid.shape

(215790, 11)

# load seqential

In [119]:
XHfc_seq, R = get_seq_Xhfc(seq_bit = 'P1_D1', h_th = 0)

Loading tag :H13_P1_D1_2_3
(896534, 8)
seq R: 16


In [120]:
XHfc_seq.shape

(327443, 11)

# get score

## get distance

In [174]:
def compute_seq_score(score, scoresRef, iSs = None, plt_val = False):
    from scipy.spatial import KDTree
    scoreA_ = np.zeros(len(score))
    if iSs is None: iSs = np.arange(scoresRef.shape[-1])
    for iS in iSs:
        dist_,inds_ = KDTree(scoresRef[:,[iS]]).query(score[:,[iS]])
        scoreA_+=np.log((inds_+1))-np.log(len(scoresRef))
        
    # plot distr
    if plt_val:
        scoreA = scoreA_
        #bad_igns = [ign for ign,gn in enumerate(dec.gns_names) if 'blank' in gn.lower()]
        #good_igns = [ign for ign,gn in enumerate(dec.gns_names) if 'blank' not in gn.lower()]
        #is_good_gn = np.in1d(dec.icodesN,good_igns)
        
        plt.figure()
#        kp = scoreA>th_min
        plt.hist(scoreA, bins = 30)
#         plt.hist(scoreA[(is_good_gn)&kp],density=True,bins=100,alpha=0.5,label='all genes')
#         if gene is not None:
#             is_gn = dec.icodesN==(list(dec.gns_names).index(gene))
#             plt.hist(scoreA[(is_gn)&kp],density=True,bins=100,alpha=0.5,label=gene)
#         plt.hist(scoreA[(~is_good_gn)&kp],density=True,bins=100,alpha=0.5,label='blanks');
#         plt.legend()
    return scoreA_

In [209]:
import numpy as np
from scipy.spatial import KDTree
from collections import defaultdict

def compute_min_distances_and_ranks(XComb, XSeq, scoresRef = None, plt_val = False):
    # Group by cell ID, skipping entries where cell ID is 0
    def group_by_cell(X):
        groups = defaultdict(list)
        for row in X:
            if row[-1] != 0:  # Skip cell ID 0
                groups[row[-1]].append(row)
        return groups

    groups_XComb = group_by_cell(XComb)
    groups_XSeq = group_by_cell(XSeq)

    # Create an expanded XSeq to store additional information
    XSeq_ = []#np.zeros((len(XSeq), len(XSeq[0]) + 2))

    # Process each cell
    for cell_id, spots_XSeq in groups_XSeq.items():
        if True:
            spots_XSeq = np.array(spots_XSeq)
            coords_XSeq = spots_XSeq[:, :3]  # Extract only coordinate columns

            if cell_id in groups_XComb:
                spots_XComb = np.array(groups_XComb[cell_id])
                coords_XComb = spots_XComb[:, :3]
                tree = KDTree(coords_XComb)
                D = tree.query(coords_XSeq, k=1)[0]  # Query the minimum distance
            else:
                # No matching cell in XComb, all distances are infinity
                D = np.full(len(coords_XSeq), np.nan)

            H = spots_XSeq[:, -4]#[:, :-1][:, -4]
            
            score = np.array([H,-D]).T
            
            if scoresRef is None: 
                scoresRef = score
            
            # compute rank sum score
            scoreA = compute_seq_score(score, scoresRef, plt_val = plt_val)
            
            # add score to output
            #print(f'XSeq current shape: {spots_XSeq.shape}')
            spots_XSeq_ = spots_XSeq.copy()
            spots_XSeq_[:, -3] = scoreA # -3 stores the score, originally set to -1
            XSeq_.append(spots_XSeq_)
            '''
            # Rank distances within the cell
            sorted_indices = np.argsort(distances)
            ranks = np.argsort(sorted_indices) + 1  # Ranks start from 1

            # Store the original rows from XSeq with additional distance and rank
            for i, spot in enumerate(spots_XSeq):
                expanded_XSeq[spot[0].astype(int)] = np.concatenate((spot, [distances[i], ranks[i]]))
            '''

    return np.vstack(XSeq_)

In [210]:
XComb = XHfc_best_valid[XHfc_best_valid[:, -2] == 700]
print(XComb.shape)
XSeq = XHfc_seq.copy()
print(XSeq.shape)
res = compute_min_distances_and_ranks(XComb, XSeq, plt_val = False)

(412, 11)
(327443, 11)


In [220]:
res.shape

(327443, 11)

In [218]:
def keep_best_seq_per_cell_fast(XH_f,nbest=20):
    '''
    modified based on ioMicroPostFits::keep_best_per_cell_fast
    '''
    ###sort molecules by score and keep only within cells (CELLs>0)
    XH_f_ = XH_f.copy()
    S = XH_f_[:,-3] #score
    XH_f_ = XH_f[np.argsort(S)]
    CELLs = XH_f_[:,-1].astype(int) #cell index
    keep = CELLs>0
    XH_f_ = XH_f_[keep]
    
    
    #################################
    CELLs = XH_f_[:,-1].astype(int) #cell index
    S = XH_f_[:,-3] #score
    ### find index combining cell id an redout id
    Rs = XH_f_[:,-2].astype(int) #seq readout
     
    cellsu = np.unique(CELLs)
    Rsu = np.unique(Rs)
    MR = np.max(Rsu)+1
    mix_index = CELLs*MR+Rs ### unique index combining cell and readout info
    
    #### find the inverse and prealocate space that you fill cyclically
    miu,rinv = np.unique(mix_index,return_inverse=True)
    miu_,cts_ = np.unique(mix_index,return_counts=True)
    XH_f_pruned = np.zeros([len(miu),nbest,XH_f.shape[1]],dtype=XH_f.dtype)+np.nan
    ind_best = np.argsort(np.argsort(rinv))%nbest
    XH_f_pruned[rinv,ind_best,:]=XH_f_######
    XH_f_pruned = XH_f_pruned.reshape([-1,XH_f_pruned.shape[-1]])
    good = ~np.isnan(XH_f_pruned)[:,0]
    XH_f_pruned = XH_f_pruned[good]
    return XH_f_pruned

In [219]:
res_best = keep_best_seq_per_cell_fast(res,nbest=5)

In [221]:
res_best.shape

(1075, 11)